In [ ]:
import pandas as pd
df = pd.read_csv("df_neurips_limitation_and_OR_with_cited_data.csv")

In [ ]:
!python3 -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip3 -q install openai

In [ ]:
# another openai account (original gmail)
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

### Methodology critic, data critic, assumption critic, master cooridinator

In [ ]:
import pandas as pd
import asyncio
from agents import Agent, Runner

import nest_asyncio
nest_asyncio.apply()

await main()


# 1) Define your agents once at module-scope
methodology_agent = Agent(
    name="MethodologyCritic",
    instructions="""
You are a methodology critic.
Analyze the METHODOLOGY of this scientific article and list 3 key limitations:
Focus on sample size, control groups, reproducibility, and bias.
Return bullet points.
""",
    model="gpt-4o-mini",
)

data_agent = Agent(
    name="DataCritic",
    instructions="""
You are a data critic.
Analyze the DATA in this article and list 3 limitations:
Focus on dataset size, missing variables, measurement errors.
Return bullet points.
""",
    model="gpt-4o-mini",
)

assumptions_agent = Agent(
    name="AssumptionsCritic",
    instructions="""
You are an assumptions critic.
Critique the ASSUMPTIONS in this article:
Focus on untested premises and generalization issues.
Return bullet points.
""",
    model="gpt-4o-mini",
)

master_agent = Agent(
    name="MasterCoordinator",
    instructions="""
You are an expert synthesizer.
Combine these three lists into a cohesive 'Limitations' section for a scientific paper:
1) Methodology

Use an academic tone and avoid redundancy.
""",
    model="gpt-4o-mini",
)


# 2) Helper to run all agents for one row
async def generate_for_row(row):
    # Build the full text from your six columns
    sections = [
        row["df_Abstract"],
        row["df_Introduction"],
        row["df_Related_Work"],
        row["df_Methodology"],
        row["df_Dataset"],
        row["df_Conclusion"],
    ]
    article_text = "\n\n".join([s for s in sections if isinstance(s, str) and s.strip()])

    # 2a) Run the three critics in parallel
    meth_task  = Runner.run(starting_agent=methodology_agent, input=article_text)
    data_task  = Runner.run(starting_agent=data_agent,       input=article_text)
    assum_task = Runner.run(starting_agent=assumptions_agent, input=article_text)

    meth_res, data_res, assum_res = await asyncio.gather(meth_task, data_task, assum_task)

    # 2b) Prepare the input for the master synthesizer
    combined = (
        f"Methodology:\n{meth_res.final_output}\n\n"
        f"Data:\n{data_res.final_output}\n\n"
        f"Assumptions:\n{assum_res.final_output}"
    )
    master_res = await Runner.run(starting_agent=master_agent, input=combined)

    # 2c) Return both final text and the four input-lists
    return {
        "limitations":      master_res.final_output,
        "input_logs": {
            "methodology": meth_res.to_input_list(),
            "data":        data_res.to_input_list(),
            "assumptions": assum_res.to_input_list(),
            "master":      master_res.to_input_list(),
        }
    }


# 3) Main entrypoint: load df, process each row, write back to df
async def main():
    # df = pd.read_csv("your_dataset.csv")  # or however you already have it in memory

    all_results = []
    for _, row in df2.iterrows():
        print("yes")
        result = await generate_for_row(row)
        print("result", result)
        all_results.append(result)

    # unpack into new columns
    df2["LLM_Limitations"] = [r["limitations"] for r in all_results]
    df2["LLM_InputLogs"]   = [r["input_logs"]  for r in all_results]

    # df2.to_csv("with_limitations.csv", index=False)
    print("Done! Wrote with LLM_Limitations and LLM_InputLogs.")



In [ ]:
# 1) Define your agents once at module-scope
methodology_agent = Agent(
    name="MethodologyCritic",
    instructions="""
You are a methodology critic.
Analyze the METHODOLOGY of this scientific article and list 3 key limitations:
Focus on sample size, control groups, reproducibility, and bias.
Return bullet points.
""",
    model="gpt-4o-mini",
)

master_agent = Agent(
    name="MasterCoordinator",
    instructions="""
You are an expert synthesizer.
Combine these three lists into a cohesive 'Limitations' section for a scientific paper:
1) Methodology

Use an academic tone and avoid redundancy.
""",
    model="gpt-4o-mini",
)


In [ ]:
# Methodology Critic Function
def critique_methodology(text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Your model name
        messages=[
            {"role": "system", "content": "You are a introduction critic. Analyze the introduction and list 3-5 limitations. Focus on sample size, controls, and bias. Return bullet points."},
            {"role": "user", "content": text}
        ],
        temperature=0.3  # Less creative, more factual
    )
    return response.choices[0].message.content

# Process each row
for index, row in df2.iterrows():
    # print(f"\nAnalyzing Paper {row['paper_id']}:")
    print(row['df_Introduction'])
    limitations = critique_methodology(row['df_Introduction'][index])
    df2.at[index, 'limitations'] = limitations
    print("--------------------")
    print(limitations)


### LLM agents to generate limitations (MARG: Agents: Experiment, Clarity, and Impact)

In [ ]:
import os
from openai import OpenAI
import time

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Define which sections to pull in for every row
SECTIONS = ["df_Abstract", "df_Introduction", "df_Related_Work",
            "df_Methodology", "df_Dataset", "df_Experiment_and_Results", "df_Conclusion"]

# Define a helper that streams one prompt and returns the full text
def run_critic(prompt: str) -> str:
    summary = ""
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
        temperature=0
    )
    for chunk in stream:
        summary += chunk.choices[0].delta.content or ""
    return summary.strip()

generated_limitations = []

for idx in range(len(df_lim)):
    # build a list of “Section:\ncontent” strings
    pieces = []
    for sec in SECTIONS:
        col = sec.replace(" ", "_")    # e.g. "Related Work" → "Related_Work"
        if pd.notna(df_lim.loc[idx, col]):
            pieces.append(f"{sec}:\n{df_lim.loc[idx, col]}")

    text_blob = "\n\n".join(pieces)

    experiments_lim = run_critic(
        '''Critically analyze the experimental methodology in this study. Highlight any potential biases, uncontrolled variables,
        or limitations in the experimental design. Suggest improvements to enhance reproducibility, statistical power, or alignment
        with best practices in [your field, e.g., 'neuroscience' or 'nanomaterials']. Focus on rigor without undermining the study’s
        core findings.\n\n'''
        + text_blob
    )
    clarity_lim = run_critic(
        '''Rewrite the [Results/Discussion] section to improve clarity for a multidisciplinary audience. Avoid jargon, use active voice,
        and emphasize logical flow. Ensure each paragraph transitions smoothly, with clear topic sentences and concise takeaways.
        Maintain technical accuracy while prioritizing accessibility. \n\n'''
        + text_blob
    )
    impact_lim = run_critic(
        '''Articulate the broader implications of these findings for [field/industry/society]. Draft a compelling 'Impact Statement'
        (3–5 sentences) that highlights: (1) How this work advances current knowledge, (2) Potential real-world applications,
        and (3) Future research directions. Avoid hyperbole; anchor claims in the data.:\n\n'''
        + text_blob
    )

    # 3) master coordinator to fuse them
    coord_prompt = (
        "You are a **Master Coordinator**.  You will be given three lists of sub-limitations.\n\n"
        f"**Experiment Critic**:\n{experiments_lim}\n\n"
        f"**Clarity Critic**:\n{clarity_lim}\n\n"
        f"**Impact Critic**:\n{impact_lim}\n\n"
        "Please synthesize these into a single, coherent **final** set of limitations, "
        "organizing them thematically and removing duplicates."
    )
    final_lim = run_critic(coord_prompt)

    generated_limitations.append({
        # "Experiment": experiments_lim,
        # "Clarity": clarity_lim,
        # "Impact": impact_lim,
        "final": final_lim
    })
    time.sleep(1)  # to avoid hitting rate limits


In [ ]:
# Extract the 'final' value from each dict and build a DataFrame
final_values = [d.get("final", "") for d in generated_limitations]
df_generated_limitations_2 = pd.DataFrame(final_values, columns=["generated_limitations_1"])

In [ ]:
# convert list to string and split
def process_single_limitation(limitation_text: str) -> list[str]:
    """
    Split the text on '**' and return the segments
    that occur before each '**'.
    """
    parts = limitation_text.split("**")
    # parts at even indices (0,2,4,…) are the “previous” segments
    prev_texts = [
        part.strip()
        for idx, part in enumerate(parts)
        if idx % 2 == 0    # even indices
           and part.strip()  # non-empty
    ]
    return prev_texts

# Apply to your DataFrame column
df_generated_limitations_2["generated_limitations_1"] = (
    df_generated_limitations_2["generated_limitations_1"]
    .apply(process_single_limitation)
)


In [ ]:
# convert string to list
import ast

# This will parse the string "[...]" into a real list object
df_generated_limitations_2['generated_limitations_1'] = (
    df_generated_limitations_2['generated_limitations_1']
      .astype(str)               # ensure it’s a string
      .apply(ast.literal_eval)   # safely evaluate Python literal
)


In [ ]:
import ast
import pandas as pd

def enumerate_and_filter(cell):
    """
    Given a cell that is either:
      - A Python list of strings, or
      - A string repr of such a list,
    this will:
      1. turn it into a list of sublists,
      2. remove any sublist equal to ['-'],
      3. prefix each remaining sublist's string with its 1-based index,
      4. return a new list-of-lists.
    """
    # 1) Parse string repr if necessary
    if isinstance(cell, str):
        try:
            lst = ast.literal_eval(cell)
        except Exception:
            # not a literal list → treat the entire cell as one string
            lst = [cell]
    else:
        lst = cell

    # 2) Ensure list-of-lists
    lol = []
    for item in lst:
        if isinstance(item, list):
            lol.append(item)
        else:
            # assume it's a bare string
            lol.append([str(item)])

    # 3) Filter out ['-'] sublists
    filtered = [sub for sub in lol if not (len(sub) == 1 and sub[0].strip() == "-")]

    # 4) Enumerate: prefix each sublist’s only element with "i. "
    enumerated = [[f"{i+1}. {sub[0]}"] for i, sub in enumerate(filtered)]

    return enumerated

# Example usage on your DataFrame
df_generated_limitations_2['generated_limitations_1'] = (df_generated_limitations_2['generated_limitations_1'].apply(enumerate_and_filter))

# Remove the first sublist in each list-of-lists
df_generated_limitations_2['generated_limitations_1'] = (df_generated_limitations_2['generated_limitations_1']
    .apply(lambda lol: lol[1:] if isinstance(lol, list) and len(lol) > 0 else [])
)


In [ ]:
def process_single_limitation(limitation_text):
    # Split into different limitations (separated by \n\n)
    limitations = limitation_text.split('\n\n')
    processed_limitations = []

    for limitation in limitations:
        # Remove numbering (e.g., "1. **Limited Literature Review**" → "**Limited Literature Review**")
        cleaned_limitation = limitation.split('. ', 1)[-1] if '. ' in limitation else limitation

        # Split into sentences (using '.')
        sentences = [s.strip() for s in cleaned_limitation.split('.') if s.strip()]

        if sentences:
            processed_limitations.append(sentences)

    return processed_limitations

# df_generated_limitations_2['generated_limitations_1'] = df_generated_limitations_2['generated_limitations_1'].apply(process_single_limitation)
df_lim['Lim_and_OR_ground_truth_final'] = df_lim['Lim_and_OR_ground_truth_final'].apply(process_single_limitation)

In [ ]:
# add numbering of LLM generated limitations
def add_numbering_to_limitations(list_of_lists):
    if not isinstance(list_of_lists, list):
        return list_of_lists  # Skip if not a list

    numbered_list = []
    for idx, sublist in enumerate(list_of_lists, start=1):
        if sublist:  # Ensure sublist is not empty
            # Add numbering to the first element of the sublist
            numbered_sublist = [f"{idx}. {sublist[0]}"] + sublist[1:]
            numbered_list.append(numbered_sublist)
    return numbered_list

# Apply to the column (modifies existing column)
df_lim['Lim_and_OR_ground_truth_final'] = df_lim['Lim_and_OR_ground_truth_final'].apply(add_numbering_to_limitations)
# df_generated_limitations_2['generated_limitations_1']  = df_generated_limitations_2['generated_limitations_1'] .apply(add_numbering_to_limitations)

In [ ]:
# remove future work
import re

def remove_future_entries(entries):
    """
    Given a list of lists (where each sub-list contains strings),
    return a new list omitting sublists where the first item contains
    'future' inside **double asterisks** (case-insensitive).
    """
    filtered = []
    for sublist in entries:
        if isinstance(sublist, list) and len(sublist) > 0:
            first_item = sublist[0]
            # Check if 'future' appears inside **...** (case-insensitive)
            if not re.search(r'\*\*.*future.*\*\*', first_item, re.IGNORECASE):
                filtered.append(sublist)
        else:
            filtered.append(sublist)  # Keep non-list entries as-is
    return filtered

# Apply to the DataFrame column
df_generated_limitations_2["generated_limitations_1"] = df_generated_limitations_2["generated_limitations_1"].apply(
    lambda lst: remove_future_entries(lst) if isinstance(lst, list) else lst
)

# remove future work
import re

def remove_future_entries(entries):
    """
    Given a list of lists (where each sub-list contains strings),
    return a new list omitting sublists where the first item contains
    'future' inside **double asterisks** (case-insensitive).
    """
    filtered = []
    for sublist in entries:
        if isinstance(sublist, list) and len(sublist) > 0:
            first_item = sublist[0]
            # Check if 'future' appears inside **...** (case-insensitive)
            if not re.search(r'\*\*.*future.*\*\*', first_item, re.IGNORECASE):
                filtered.append(sublist)
        else:
            filtered.append(sublist)  # Keep non-list entries as-is
    return filtered

# Apply to the DataFrame column
df_lim["Lim_and_OR_ground_truth_final"] = df_lim["Lim_and_OR_ground_truth_final"].apply(
    lambda lst: remove_future_entries(lst) if isinstance(lst, list) else lst
)

def remove_here_are_the(entries):
    """
    Given a list of lists (where each sub-list contains strings),
    return a new list omitting any sub-list that starts with "1. Here are the"
    (case-insensitive and whitespace-tolerant).
    """
    filtered = []
    for sublist in entries:
        if isinstance(sublist, list) and len(sublist) > 0:
            first_item = sublist[0].strip().lower()  # Clean whitespace and lowercase
            if not first_item.startswith("1. here are the"):
                filtered.append(sublist)
        else:
            filtered.append(sublist)  # Keep non-list entries as-is
    return filtered

# Apply to every row in the DataFrame
df_lim["Lim_and_OR_ground_truth_final"] = df_lim["Lim_and_OR_ground_truth_final"].apply(
    lambda lst: remove_here_are_the(lst) if isinstance(lst, list) else lst
)